# Database Cracking Demo


```
 Copyright (C) 2017  LSBD Adaptive Databases Group
 
Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

      http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.
```

## Reference:

Idreos et. al. - Database Cracking - ICDE 2007

In [1]:
def crack_in_three(col, low, high, c1, c2):
    """
    Cracks a column partition into three subparitions
    and return te position of the new partition points
    """
    i = low  # keeps track of the initial lower bound
    j = high - 1 # keeps track of the initial upper bound
    swaps = 0 # keeps track of the number of swaps performed
    
    while col[j] >= c2 and j > i:
        j -= 1
        
    # from this point k is the lower bound
    # i is used as a supporting variable
    k = j
    
    while col[k] > c1 and k > i:
        if col[k] >= c2:
            col[j], col[k] = col[k], col[j]
            swaps += 1
            j -= 1
        k -= 1
    
    while i < k:
        if col[i] < c1:
            i += 1
        else:
            col[i], col[k] = col[k], col[i]
            swaps += 1
            while col[k] > c1 and k > i:
                if col[k] >= c2:
                    col[j], col[k] = col[k], col[j]
                    swaps += 1
                    j -= 1
                k -= 1
                
    return ((k, j), swaps)

## CrackInThree algorithm

Partitions the column so that all values in the selected range end up
clustered together

**Example:**

In [2]:
import random

# Numbers from 0 to 19
column = list(range(20))
random.shuffle(column)
print(column)

# Crack and reurn the answer to the user
print(crack_in_three(column, 0, len(column), 8, 16))
print(column)

[0, 12, 18, 2, 14, 1, 15, 9, 7, 19, 10, 4, 6, 5, 16, 13, 3, 11, 8, 17]
((8, 15), 13)
[0, 3, 5, 2, 6, 1, 4, 7, 8, 12, 10, 9, 15, 14, 11, 13, 19, 16, 18, 17]


## Cracker SELECT

Each query looks for the lower and upper boundary in the cracker index
and then cracks the partitions in the middle, if they aren't already in the
smallest size.

In [3]:
idx_min = None
idx_max = None
part_num = 0

def cracker_select(col, idx, low, high):
    """
    TODO
    """
    part_num += 1
    if len(idx) == 0:
        ans = crack_in_three(col, 0, len(col), low, high)
        idx_min = ans[0][0]
        idx.insert(idx_min)
        idx_max = ans[0][1]
        idx.insert(idx_max)
        return idx_max - idx_min
    else:
        first_part = 0 if low < idx_min else idx.floor_item(low)
        last_part = part_num if high > idx_max else idx.floor_item(high)
        

In [ ]:
import bintrees

# cracker column (hundred million values)
column = list(range(100))
random.shuffle(column)

# cracker index. keeps track of the positions of the cracks
cracker_index = bintrees.AVLTree()